In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

Using TensorFlow backend.
/home/steve/venv_nsfw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/steve/venv_nsfw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/steve/venv_nsfw/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/steve/venv_nsfw/lib/python3.6/site-packages/tensorflo

In [2]:
data = pd.read_csv('train.csv')
aug_data = pd.read_csv('augmented_train.csv')

In [3]:
for i in range(data.shape[0]):
    data['HS'].iloc[i] = data['HS'].iloc[i].replace('*','')
    data['Avg'].iloc[i] = data['Avg'].iloc[i].replace('-','0')

/home/steve/venv_nsfw/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
data.tail(5)

,PLAYER,Mat,Inns,NO,2018_Runs,HS,Avg,BF,SR,100,50,4s,6s,2019_Runs
95,Virat Kohli,14,14,3,530,92,48.18,381,139.10,0,4,52,18,488
96,Washington Sundar,7,6,3,65,35,21.66,38,171.05,0,0,5,4,64
97,Wriddhiman Saha,11,10,2,122,35,15.25,102,119.60,0,0,17,1,115
98,Yusuf Pathan,15,13,4,260,45,28.88,200,130.00,0,0,22,11,296
99,Yuvraj Singh,8,6,0,65,20,10.83,73,89.04,0,0,6,2,67


In [16]:
aug_data.head(10)

,PLAYER,class,form,2018_Runs,2019_Runs
0,Aaron Finch,1.0,30.000000,134,160
1,AB de Villiers,2.0,73.584906,480,424
2,Abhishek Sharma,0.0,38.095238,63,63
3,Ajinkya Rahane,2.0,78.787879,370,396
4,Alex Hales,0.0,63.030303,148,165
5,Ambati Rayudu,3.0,81.381958,602,521
6,Andre Russell,2.0,38.526912,316,353
7,Andrew Tye,0.0,84.210526,32,19
8,Axar Patel,0.0,30.000000,80,80
9,Ben Cutting,0.0,44.444444,96,90


In [27]:
for index,row in data.iterrows():
    player = row['PLAYER']
    if row['Inns'] > 8 and row['2018_Runs'] >400:
        cl = 3
    elif row['Inns'] > 8 and row['2018_Runs'] >200:
        cl = 2
    elif row['Inns'] > 8 and row['2018_Runs'] >100:
        cl = 1
    else:
        cl = 0
    form = (row['4s']*4 + row['6s']*6)*100/(row['2018_Runs'])
        
    
    aug_data['class'][aug_data['PLAYER']==player] = int(cl)
    if cl == 0:
        aug_data['form'][aug_data['PLAYER']==player] = 20
    else:
        aug_data['form'][aug_data['PLAYER']==player] = round(form,1)
        
    

/home/steve/venv_nsfw/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/steve/venv_nsfw/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/steve/venv_nsfw/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [28]:
aug_data.to_csv('augmented_train.csv',index = False)

In [29]:
data = pd.read_csv('test.csv')
aug_data = pd.read_csv('augmented_test.csv')

In [30]:
for i in range(data.shape[0]):
    data['HS'].iloc[i] = data['HS'].iloc[i].replace('*','')
    data['Avg'].iloc[i] = data['Avg'].iloc[i].replace('-','0')

In [33]:
for index,row in data.iterrows():
    player = row['PLAYER']
    if row['Inns'] > 8 and row['2019_Runs'] >400:
        cl = 3
    elif row['Inns'] > 8 and row['2019_Runs'] >200:
        cl = 2
    elif row['Inns'] > 8 and row['2019_Runs'] >100:
        cl = 1
    else:
        cl = 0
    form = (row['4s']*4 + row['6s']*6)*100/(row['2019_Runs'])
        
    
    aug_data['class'][aug_data['PLAYER']==player] = int(cl)
    if cl == 0:
        aug_data['form'][aug_data['PLAYER']==player] = 20
    else:
        aug_data['form'][aug_data['PLAYER']==player] = round(form,1)
        

/home/steve/venv_nsfw/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/steve/venv_nsfw/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/steve/venv_nsfw/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [34]:
aug_data.to_csv('augmented_test.csv',index = False)